หน้าที่หลัก: เปลี่ยนกลุ่มไฟล์ (TIFF) ให้กลายเป็น (NetCDF) 

Find & Filter Files (✅ ทำเสร็จแล้ว):

- ใช้ glob กวาดหาไฟล์ทั้งหมด

- ใช้ Dictionary Logic คัดเลือกเฉพาะไฟล์ล่าสุด (ถ้ามี _fixed ให้เอาตัวนั้น)

- ผลลัพธ์: ได้ List รายชื่อไฟล์ที่ถูกต้อง 360 ไฟล์

Load & Time Stamping (👉 กำลังจะทำ - Cell 3):

- อ่านไฟล์ภาพทีละรูป

- แกะ "วันที่" จากชื่อไฟล์ (เช่น 202512 -> Dec 2025)

- แปะป้ายวันที่ลงไปในข้อมูล

Concatenate (รวมร่าง):

- จับภาพ 360 ใบ มาซ้อนกันเป็นก้อนเดียว (Data Cube)

- ขนาดจะเป็น 3 มิติ: (Time, Lat, Lon)

Sanity Check (ตรวจสุขภาพ):

- เช็คว่าเวลาครบ 360 เดือนไหม? (ไม่ขาด ไม่เกิน)

- เช็คค่า min/max (เจอ -9999 หรือค่าแปลกๆ ไหม)

Clean & Export (ทำความสะอาดและบันทึก):

- เปลี่ยนค่า Error (-9999) ให้เป็น NaN

- Save เป็นไฟล์ .nc (NetCDF)

- จบงาน: ส่งไฟล์นี้ไปให้ Notebook 2 คำนวณ SPI ต่อ

In [ ]:
import glob
import os
import xarray as xr
import rioxarray
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm  # ใช้ดู Progress Bar 

In [2]:
"""
    เก็บไฟล์ฝนตรวจวัดในรูปแบบ dictionary {ชื่อไฟล์: path} 
    เช่น {'o_th202512': "path/.../o_th202512_fixed.tif"}
    ถ้าไฟล์ไหนมี suffix เป็น "_fixed" path ของไฟล์นั้นจะถูก overwrite ด้วยตัวที่มี suffix
    เช่น โค้ดจะเก็บ path ของตัว o_th202512_fixed.tif และไม่เก็บ o_th202512.tif ลง dictionary
"""

ROOT_PATH = Path(r"D:\HII\drought_forecasting")
DATA_PATH = ROOT_PATH / "data"
RAW_DATA_PATH = DATA_PATH / "raw"
RAIN_OBS_PATH = RAW_DATA_PATH / "rain_obs_monthly"

rain_dict = {
    p.stem.replace("_fixed", ""): p                 # .stem จะเอาแค่ชื่อไฟล์ ตัด path และนามสกุลออก เช่น "rain_202512_fixed"
    for p in sorted(RAIN_OBS_PATH.glob("*.tif"))
}

rain_paths = sorted(list(rain_dict.values()))

rain_paths[:3]

[WindowsPath('D:/HII/drought_forecasting/data/raw/rain_obs_monthly/o_th197001.tif'),
 WindowsPath('D:/HII/drought_forecasting/data/raw/rain_obs_monthly/o_th197002.tif'),
 WindowsPath('D:/HII/drought_forecasting/data/raw/rain_obs_monthly/o_th197003.tif')]

In [ ]:
# --- Configuration ---
DATE_FORMAT = "%Y%m"           
VAR_NAME = "precipitation"  # เตรียมตัวชื่อตัวแปรให้ DataArray

data_buffer = []

# --- Raster Ingestion Pipeline ---
for file_path in tqdm(rain_paths, desc="Ingesting Raster Data"):
    
    try:
        # 1. Extract Timestamp
        # Normalize filename: remove suffix -> take last 6 chars -> parse date
        date_str = file_path.stem.replace("_fixed", "")[-6:]
        timestamp = pd.to_datetime(date_str, format=DATE_FORMAT)

        # 2. Load and Transform Raster
        # 'masked=True' automatically converts NoData values to NaN
        # Using context manager (with) ensures file handles are closed immediately
        with rioxarray.open_rasterio(file_path, masked=True) as da:
            
            # 3. Dimensionality Reduction & Expansion
            # Squeeze: Remove 'band' dimension (1, y, x) -> (y, x)
            # Expand_dims: Add 'time' dimension -> (time, y, x)
            da_processed = (
                da.squeeze(dim="band", drop=True)
                  .expand_dims(time=[timestamp])
            )
            
            da_processed.name = VAR_NAME
            
            # .copy() is essential here because 'da' is closed after 'with' block
            data_buffer.append(da_processed.copy())
            
    except Exception as e:
        print(f"⚠️ Error processing file {file_path.name}: {e}")
        continue

# --- Concatenation & Finalization ---
if data_buffer:
    # Stack all time steps along the 'time' dimension
    obs_rain_da = xr.concat(data_buffer, dim="time", join="override")
    
    # Ensure time series is strictly monotonic
    obs_rain_da = obs_rain_da.sortby("time")
    
    # แปลงเป็น Dataset
    obs_rain_ds = obs_rain_da.to_dataset()
    
# Quick Inspection
    print(f"✅ Pipeline Completed.")
    print(obs_rain_ds) # เช็คดูหัวข้อต้องเป็น xarray.Dataset
else:
    print("❌ No data loaded.")

Ingesting Raster Data: 100%|██████████| 665/665 [00:38<00:00, 17.15it/s]


✅ Pipeline Completed.
<xarray.Dataset> Size: 3GB
Dimensions:        (time: 665, x: 829, y: 1485)
Coordinates:
  * time           (time) datetime64[us] 5kB 1970-01-01 ... 2025-12-01
  * x              (x) float64 7kB 97.35 97.36 97.37 97.38 ... 105.6 105.6 105.6
  * y              (y) float64 12kB 20.46 20.45 20.44 20.43 ... 5.64 5.63 5.62
    spatial_ref    int64 8B 0
Data variables:
    precipitation  (time, y, x) float32 3GB nan nan nan nan ... nan nan nan nan


In [ ]:
# --- Configuration ---
PROCESSED_DATA_PATH = DATA_PATH / "processed"
NET_CDF_PATH = PROCESSED_DATA_PATH / "observed_rain_1970_2025_raw.nc"

PROCESSED_DATA_PATH.mkdir(parents=True, exist_ok=True)

# --- Save (No Compression) ---
print(f"💾 Saving Dataset to: {NET_CDF_PATH}")
obs_rain_ds.to_netcdf(NET_CDF_PATH)

print("✅ Save Completed!")

💾 Saving Dataset to: D:\HII\drought_forecasting\data\processed\observed_rain_1970_2025_raw.nc
✅ Save Completed!
